In [1]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable

In [2]:
# Hyper Parameters
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST Dataset
train_dataset = dsets.MNIST(root='../data/',
                            train=True, 
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='../data/',
                           train=False, 
                           transform=transforms.ToTensor())

Processing
Done!


In [ ]:
# OUR PROJECT!!

#https://discuss.pytorch.org/t/questions-about-imagefolder/774/6
#http://pytorch.org/docs/data.html

#https://github.com/pytorch/vision/tree/master/torchvision/models MODELS!!

train = dsets.ImageFolder(root="/Users/leighharton/Documents/MSDS/CS_6501/project_test", transform=ToTensor())

train_VIST = dsets.ImageFolder(root = 'directory', 
                               train = True,
                               transform = ToTensor())

test_VIST = dsets.ImageFolder(root = 'directory', 
                              train = False,
                              transform = ToTensor())

# x,y = data[0] # x is the first image as PIL, y is that images class label
# features = 2D, targets = 1D
train = torch.utils.TensorDataset(features, targets)

In [9]:
# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [ ]:
# OUR PROJECT!!
trainVIST_loader = torch.utils.data.DataLoader(dataset=train_VIST,
                                               batch_size=2, 
                                               shuffle=True)

testVIST_loader = torch.utils.data.DataLoader(dataset=test_VIST,
                                               batch_size=2, 
                                               shuffle=False)

In [ ]:
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [10]:
# OUR PROJECT?
# CNN Model (2 conv layer)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 96, kernel_size=11, padding=2), #in channels, out channels
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(3, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),#change?
            nn.ReLU(), #change?
            nn.MaxPool2d(3, stride=2))
        self.layer3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.layer4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.layer5 = nn.Sequential(
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.MaxPool2d(3, stride=2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
        
cnn = CNN()

In [6]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

# Train the Model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images)
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))

Epoch [1/5], Iter [100/600] Loss: 0.1569
Epoch [1/5], Iter [200/600] Loss: 0.0730
Epoch [1/5], Iter [300/600] Loss: 0.0741
Epoch [1/5], Iter [400/600] Loss: 0.0320
Epoch [1/5], Iter [500/600] Loss: 0.0859
Epoch [1/5], Iter [600/600] Loss: 0.0796
Epoch [2/5], Iter [100/600] Loss: 0.0473
Epoch [2/5], Iter [200/600] Loss: 0.1117
Epoch [2/5], Iter [300/600] Loss: 0.0183
Epoch [2/5], Iter [400/600] Loss: 0.0444
Epoch [2/5], Iter [500/600] Loss: 0.0141
Epoch [2/5], Iter [600/600] Loss: 0.0312
Epoch [3/5], Iter [100/600] Loss: 0.0197
Epoch [3/5], Iter [200/600] Loss: 0.0591
Epoch [3/5], Iter [300/600] Loss: 0.0225
Epoch [3/5], Iter [400/600] Loss: 0.0204
Epoch [3/5], Iter [500/600] Loss: 0.0101
Epoch [3/5], Iter [600/600] Loss: 0.0103
Epoch [4/5], Iter [100/600] Loss: 0.0459
Epoch [4/5], Iter [200/600] Loss: 0.0228
Epoch [4/5], Iter [300/600] Loss: 0.0155
Epoch [4/5], Iter [400/600] Loss: 0.0095
Epoch [4/5], Iter [500/600] Loss: 0.0493
Epoch [4/5], Iter [600/600] Loss: 0.0237
Epoch [5/5], Ite

In [7]:
# Test the Model
cnn.eval()  # Change model to 'eval' mode (BN uses moving mean/var).
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images)
    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Test Accuracy of the model on the 10000 test images: %d %%' % (100 * correct / total))

Test Accuracy of the model on the 10000 test images: 98 %


In [ ]:
# Extra code:
from PIL import Image
jpgimage = Image.open('/Users/leighharton/Documents/MSDS/CS_6501/mscoco/COCO_train2014_000000000025.jpg').convert('RGB')
type(jpgimage)